In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras.optimizers
import tensorflow as tf

import cv2

from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/dataset/dataset-jagung/data-excel/PenyakitDaunJagung.csv")
data.head()

# /content/drive/MyDrive/dataset/dataset-jagung/data-excel/hawar_daun.xlsx
# /content/drive/MyDrive/dataset/dataset-jagung/data-excel/bercak_daun.xlsx
# /content/drive/MyDrive/dataset/dataset-jagung/data-excel/Karatan_daun.xlsx

In [ ]:
import pandas as pd

# Path ke file Excel
path_bercak = '/content/drive/MyDrive/dataset/dataset-jagung/data-excel/bercak_daun.xlsx'
path_hawar = '/content/drive/MyDrive/dataset/dataset-jagung/data-excel/hawar_daun.xlsx'
path_karatan = '/content/drive/MyDrive/dataset/dataset-jagung/data-excel/Karatan_daun.xlsx'

# Membaca data dari masing-masing file Excel
df_bercak = pd.read_excel(path_bercak)
df_hawar = pd.read_excel(path_hawar)
df_karatan = pd.read_excel(path_karatan)

# Menggabungkan data
df_gabungan = pd.concat([df_bercak, df_hawar, df_karatan], ignore_index=True)

# Menyimpan data gabungan ke file Excel baru
output_path = '/content/drive/MyDrive/dataset/dataset-jagung/data-excel/singleObjectDetection.xlsx'
df_gabungan.to_excel(output_path, index=False)

print("Data berhasil digabungkan dan disimpan di:", output_path)

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/dataset/dataset-jagung/data-excel/singleObjectDetection.xlsx')
data.head()

In [ ]:
# prompt: total data

print(data.info())
print(data.describe())


In [ ]:
data = data[:983]
len(data)

In [ ]:
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from PIL import ImageFile
import numpy as np

# Allow loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

image_size = 224
image_list = []
invalid_images = []

# Loop through the dataset and load images
for i in tqdm(range(data.shape[0])):
    try:
        img = image.load_img('/content/drive/MyDrive/dataset/dataset-jagung/images/' + data['image'][i], target_size=(image_size, image_size, 3))
        img = image.img_to_array(img)
        img = img / 255.0
        image_list.append(img)
    except (OSError, ValueError) as e:
        print(f"Error loading image {data['image'][i]}: {e}")
        invalid_images.append(i)

# Remove invalid images from the DataFrame
data = data.drop(invalid_images).reset_index(drop=True)

# Convert the list of images to a NumPy array
X = np.array(image_list)

In [ ]:
image_list = np.array(image_list)
print(image_list)

In [ ]:
#Categorical Data
y = data[['Hawar Daun', 'Bercak Daun', 'Karatan Daun']].values
y

In [ ]:
x_train, x_testval, y_train, y_testval = train_test_split(image_list, y, test_size=0.2, random_state=42) #80% Train, 10% Validation, 10% Test
x_val, x_test, y_val, y_test = train_test_split(x_testval, y_testval, test_size=0.5, random_state=42)

In [ ]:
import keras
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
model = InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

In [ ]:
# Tambahkan lapisan tambahan ke model base Anda
x = GlobalAveragePooling2D()(model.output)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
output = Dense(3, activation='sigmoid')(x)

In [ ]:
model = keras.models.Model(inputs=model.input, outputs=output)
model.summary()

In [ ]:
# optimizer = keras.optimizers.Adam(learning_rate=0.0001)

# model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.compile(optimizer=Adam(learning_rate=0.001),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

# Tambahkan callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)


In [ ]:
# history = model.fit(x_train,
#                     y_train,
#                     epochs=20,
#                     batch_size=64,
#                     validation_data=(x_val, y_val)
#                     )

# Train model
history = model.fit(x_train,
                          y_train,
                          epochs=30,
                          batch_size=16,
                          validation_data=(x_val, y_val),
                          callbacks=[early_stopping, reduce_lr])

In [ ]:
# prompt: tampilkan # Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Asumsikan x_test dan y_test sudah tersedia

# Evaluasi model pada test set
y_pred = model.predict(x_test)
y_pred_classes = (y_pred > 0.5).astype(int)  # Konversi probabilitas menjadi label kelas

# Hitung akurasi
accuracy = accuracy_score(y_test, y_pred_classes)
print("Accuracy:", accuracy)

# Buat laporan klasifikasi
print(classification_report(y_test, y_pred_classes, target_names=['Hawar Daun', 'Bercak Daun', 'Karatan Daun']))

# Buat confusion matrix
conf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred_classes.argmax(axis=1))

# Visualisasi confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Hawar Daun', 'Bercak Daun', 'Karatan Daun'], yticklabels=['Hawar Daun', 'Bercak Daun', 'Karatan Daun'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Simpan Model Tflite
# Menyimpan model dalam format tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan model tflite ke file
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# prompt: simpan juga labels.txt nya

# Simpan labels.txt
with open('labels.txt', 'w') as f:
  f.write('Hawar Daun\n')
  f.write('Bercak Daun\n')
  f.write('Karatan Daun\n')

# Code to save the model to your Google Drive (optional)
from google.colab import drive
drive.mount('/content/drive')

!cp model.tflite /content/drive/MyDrive/
!cp labels.txt /content/drive/MyDrive/


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

# Load dan preprocess gambar
img_path = '/content/drive/MyDrive/dataset/dataset-jagung/hawar/hawar-10.jpg'
img = image.load_img(img_path, target_size=(image_size, image_size))
img = image.img_to_array(img)
img = img / 255.0

# Prediksi
proba = model.predict(img.reshape(1, image_size, image_size, 3))
classes = np.array(['Hawar Daun', 'Bercak Daun', 'Karatan Daun'])  # Gantilah sesuai dengan kelas Anda

# Gabungkan kelas dan probabilitas ke dalam satu list
class_proba = list(zip(classes, proba[0]))

# Urutkan berdasarkan probabilitas (dari tertinggi ke terendah)
class_proba_sorted = sorted(class_proba, key=lambda x: x[1], reverse=True)

# Tampilkan hasil peringkat
print("Hasil Prediksi (Diurutkan berdasarkan Probabilitas):")
for i, (class_name, probability) in enumerate(class_proba_sorted):
    print(f"{i+1}. {class_name}: {probability*100:.2f}%")

# Tampilkan gambar
plt.imshow(img)
plt.axis('off')  # Sembunyikan sumbu
plt.show()

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import os
import random

# Muat model TFLite
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Dapatkan input dan output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Path dataset dan ambil daftar file gambar dari semua subfolder
dataset_path = '/content/drive/MyDrive/dataset/dataset-jagung/'
image_files = []
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith('.jpg'):
            image_files.append(os.path.join(root, file))

# Pilih 10 gambar secara acak
sample_files = random.sample(image_files, 10)

# Fungsi untuk memproses gambar dan melakukan prediksi
def predict_image(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(image_size, image_size))
    img = image.img_to_array(img)
    img = img / 255.0

    # Predict
    interpreter.set_tensor(input_details[0]['index'], img.reshape(1, image_size, image_size, 3).astype(np.float32))
    interpreter.invoke()
    proba = interpreter.get_tensor(output_details[0]['index'])[0]

    # Apply threshold
    threshold = 0.5
    classes = np.array(data.columns[2:])
    top_classes = [classes[i] for i in range(len(proba)) if proba[i] > threshold]
    top_proba = [proba[i] for i in range(len(proba)) if proba[i] > threshold]

    return img, top_classes, top_proba

# Setup plot
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
axes = axes.flatten()

# Predict and plot images
for ax, file_name in zip(axes, sample_files):
    img_path = file_name
    img, top_classes, top_proba = predict_image(img_path)

    ax.imshow(img)
    ax.axis('off')
    result_str = ', '.join([f"{cls} ({p:.2f})" for cls, p in zip(top_classes, top_proba)])
    ax.set_title(result_str, fontsize=10)

plt.tight_layout()
plt.show()
